# Acordo de chaves para comunicação confidencial e autenticada

Construção de uma sessão síncrona de comunicação segura entre dois agentes (o Emitter e o Receiver), combinando os seguintes elementos constituintes:
- a cifra simétrica TAES (AES na sua versão tweak) usando autenticação do criptograma em cada superbloco; 
- o protocolo de acordo de chaves Diffie-Hellman com verificação da chave, e autenticação dos agentes através do esquema de assinaturas DSA.

## Connection

In [1]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        self.lproc = Process(target = left, args=(left_end,))
        self.rproc = Process(target = right, args=(right_end,))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        genDSAKeyPair("emitter")
        genDSAKeyPair("receiver")
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

## Primitivas criptográficas

### Chaves de longa duração

A geração de chaves de longa duração é implementada pela função `genDSAKeyPair` que gera um par de chaves para assinaturas DSA e guarda cada uma em ficheiros **PEM**.

A chave privada é armazenada com recurso ao formato **PKCS8** e cada par de chaves é gerado apenas na criação da conexão **i.e.** na chamada do construtor da classe `Connection`, mantendo-se constante para ligações estabelecidas na mesma instância da classe.

In [2]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import dsa

def genDSAKeyPair(fname):
    privKeyF = open(fname+"_priv.pem", 'wb')
    pubKeyF = open(fname+"_pub.pem", 'wb')
   
    privKey = dsa.generate_private_key(
         key_size=3072,
         backend=default_backend()
    )
    pubKey = privKey.public_key()
    
    privKeyDump = privKey.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.BestAvailableEncryption(fname.encode("ascii")) # Store encrypted key
    )
    pubKeyDump = pubKey.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    privKeyF.write(privKeyDump)
    privKeyF.close()
    pubKeyF.write(pubKeyDump)
    pubKeyF.close()

As funções que se seguem: `verifyTag`, `extractMessage`, `sign_message`, `verify_signature` e `xor_bytes`; estão respetivamente descritas e servem de funções auxiliares durante o processo de comunicação.

Note-se que o tempo de geração dos parâmetros do protocolo de acordo de chaves Diffie-Hellman, recorrendo ao método generate_parameters( ), era relativamente elevando, tendo-se consequentemente optado por fixar os mesmo, como se verifica de seguida.

In [3]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import os

# DH Parameters
P = 99494096650139337106186933977618513974146274831566768179581759037259788798151499814653951492724365471316253651463342255785311748602922458795201382445323499931625451272600173180136123245441204133515800495917242011863558721723303661523372572477211620144038809673692512025566673746993593384600667047373692203583
G = 44157404837960328768872680677686802650999163226766694797650810379076416463147265401084491113667624054557335394761604876882446924929840681990106974314935015501571333024773172440352475358750668213444607353872754650805031912866692119819377041901642732455911509867728218394542745330014071040326856846990119719675

def verify_tag(key, tag, tweak_iv, ctxt):
    #TODO: verify ctxt with tag
    pass

def decipher_message(key, tweak_iv, ctxt):
    """ Extracts the plain text from a cryptogram resulting from a tweakable cipher
        where ptxt = dec(ctxt ^ hash(tweak)) ^ hash(tweak)
    """
    tweak_iv = list(tweak_iv)
    bs = len(tweak_iv)*2
    parity = bytes([0]*bs)
    ptxt = bytes()
    
    for i in range(0, len(ctxt), bs):
        block_index = i // bs
        lower_tweak = list((block_index*2).to_bytes(bs//2, 'big'))         # convert i*2 to binary format
        block_tweak = create_block_hash(tweak_iv, lower_tweak)             # tweak digest
        block_in = xor_bytes(ctxt, block_tweak, i, bs)
        cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).decryptor()
        ptxt_block = xor_bytes(cipher.update(block_in) + cipher.finalize(), block_tweak, 0, bs)
        ptxt += ptxt_block
        parity = xor_bytes(ptxt_block, parity, 0, bs)           # update parity
    
    #TODO: verify tag
    
    return ptxt

def sign_message(priv_key, msg):
    """ Signs a particular message with a given private key
    """
    signature = priv_key.sign(msg,hashes.SHA256())
    return {'sig': signature, 'msg' : msg}

def verify_signature(pub_key, signature, msg):
    """ Verify that the private key associated with a particular public key was used to sign that particular message
    """
    try:
        pub_key.verify(signature,msg,hashes.SHA256())
        return True
    except:
        return False
    
def pad_message(msg, block_size):
    if (len(msg) % block_size) != 0:
        pad_size = block_size - (len(msg) % block_size)
        msg = bytes(list(msg).extend([0]*pad_size))
    return msg
    
def parity_block(tweak_iv, msg_len, bs, parity):
    lower_parity = list((msg_len * 16 + 1).to_bytes(bs//2, 'big'))
    block_tweak = create_block_hash(tweak_iv, lower_parity)
    tag_in = xor_bytes(parity, block_tweak, 0, bs)
    return (block_tweak, tag_in)

def create_block_hash(tweak_iv, lower_tweak):
    tweak_iv.extend(lower_tweak)
    block_tweak = bytes(tweak_iv)
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(block_tweak)
    return (digest.finalize())[16:]
    
def encrypt_message(msg, tweak_iv, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).encryptor()
    tweak_iv_l = list(tweak_iv)
    bs = len(tweak_iv)*2
    # pad_msg = pad_message(msg, bs)
    pad_msg = msg
    parity = bytes([0]*bs)
    ctxt = bytes()
    
    for i in range(0, len(pad_msg), bs):
        block_index = (i // bs)
        parity = xor_bytes(pad_msg, parity, i, bs)              # update parity
        lower_tweak = list((block_index*2).to_bytes(bs//2, 'big'))         # convert i*2 to binary format
        block_tweak = create_block_hash(tweak_iv_l, lower_tweak)    # tweak digest
        block_in = xor_bytes(pad_msg, block_tweak, i, bs)
        ctxt_block = xor_bytes(cipher.update(block_in), block_tweak, 0, bs)
        ctxt += ctxt_block
    
    ctxt += cipher.finalize()
    
    # Compute authentication tag
    block_tag = parity_block(tweak_iv_l, len(pad_msg), bs, parity)
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).encryptor()
    tag = xor_bytes(cipher.update(block_tag[1]) + cipher.finalize(), block_tag[0], 0, bs)
    
    payload = {'content': ctxt, 'tag': tag, 'tweak_iv': tweak_iv}
    
    return payload
    
    
def xor_bytes(m1, m2, offset, bs):
    """ Xor bytes
    """
    xored_res = []
    for i in range(0, bs):
        xored_res.append(m1[offset+i] ^ m2[i])
    return bytes(xored_res)

## Emitter

O Emitter começa por gerar a chave partilhada para cifrar as mensagens trocadas com o Receiver, conforme o protocolo de acordo de chaves *Diffie-Hellman*, armazenando-a numa variável local. Adicionalmente, e para prevenir a realização de ataques Man-In-The-Middle, que violam a autenticidade das mensagens trocadas durante o protocolo, este recorre a chaves para assinar e verificar as assinaturas DSA, que estavam armazenadas em ficheiros com nomes alusivos a cada uma dessas operações.

Com recurso às funções `sign_message` e `verify_signature` efetua-se nomeadamente a assinatura e a respetiva verificação da mesma. Sendo que a primeira recebe como parâmetros a chave privada usada para assinar (`sign_key`) e a mensagem `public_key_pem` e a segunda recebe a chave pública (`ver_key`), a assinatura esperada (`sig`) e a mensagem à qual esta assinatura se refere (`msg`).

Se durante no protocolo de troca de chaves não ocorrer nenhuma falha ao nível da verificação da troca das mesmas, sinalizada pela mensagem b"0", determina-se a chave acordada no  protocolo DH (`s_key`). Posteriormente, recorrendo a esta,`s_key`, e à função auxiliar `deriveKey` gera-se a chave da cifra (`c_key`) e a chave do HMAC (`h_key`).

Seguidamente gerou-se o `tweak`(*nonce*) como uma combinação do tempo no instante atual, em segundos, com 8 *bytes* aleatórios.

Posteriormente calcula-se o hash do valor do tweak (`tweak_h`) e ainda o HMAC de `header + content_new + iv + tweak_h` que é armazenado no campo `m_hmac`. Tal permite que a autenticação seja feita via uma *tag* que deve ser utilizada no momento de decifragem dos dados e, caso não corresponda ao valor esperado, resulta numa exceção.

Para além da mensagem trocada entre os dois interveniente são ainda enviados dados que permitem a sua decifragem
por parte do `Receiver` nomeadamente a *tag* esperada para decifrar os mesmos, o *hash* do tweak utilizado e ainda os metadados previamente autenticados:

payload = {'header': header, 'content': content_new, 'm_hmac': m_hmac, 'tweak_h': tweak_h}

In [4]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.asymmetric import dh
import os, time

def Emitter(conn):
    try:
        # Load long-term keys
        sign_key_dump = open("emitter_priv.pem", "rb").read()
        sign_key = load_pem_private_key(sign_key_dump, "emitter".encode("ascii"), backend=default_backend())
        
        ver_key_dump = open("receiver_pub.pem", "rb").read()
        ver_key = load_pem_public_key(ver_key_dump, backend=default_backend()) 
        
        # Session keys
        parameters = dh.DHParameterNumbers(P,G).parameters(backend=default_backend())
        priv_key = parameters.generate_private_key()
        public_key_pem = priv_key.public_key().public_bytes(
                                        encoding=serialization.Encoding.PEM,
                                        format=serialization.PublicFormat.SubjectPublicKeyInfo
                                        )      

        payload = sign_message(sign_key, public_key_pem)
        conn.send(payload)                
        
        payload = conn.recv()
        if verify_signature(ver_key, payload['sig'], payload['msg']) and payload['msg'] != b"0":
            g_y = load_pem_public_key(payload['msg'], backend=default_backend())
            #Generate shared secret
            s_key = priv_key.exchange(g_y)[:16]
            conn_alive = True
            
            while conn_alive:
                content = '1234567890123456'.encode("ascii")
                # content = input("Enter a message to send to Receiver (to close the connection send 0): ")
                
                # Set tweak (nonce) to current time since epoch in seconds + 4 byte random value
                tweak = str(time.time()).encode("ascii")[-4:]+os.urandom(4)
            
                payload = encrypt_message(content, tweak, s_key)
            
                conn.send(payload)
                resp = conn.recv()
                conn_alive = (resp != b"0")
                
        else:
            if payload['msg'] != b"0":
                payload = sign_message(sign_key, b"0")
                conn.send(payload)
            else:
                print("A verificação na troca de chaves falhou!")
        conn.close()
    except Exception as e:
        print("Emitter:" + e)
        print("Emitter: Error")

## Receiver

O Receiver, após a troca de chaves, e a cada mensagem recebida, efetua uma verificação da autenticidade e integridade da mesma com recurso à função `verifyHMAC` que recebe como parâmetro a chave usada no cálculo do MAC (`h_key`), o MAC esperado (`m_hmac`) e o *input* sobre o qual este deve ser calculado (`header+ctxt+tweak_h`).

Caso a mensagem recebida não tenha sido modificada, o Receiver procede ao processo de decifragem. Dado tratar-se de
uma mensagem cifrada com **TAES** é necessário manipular o criptograma de modo a obter a mensagem original, sendo útil ter em conta a seguinte expressão para o processo de decifragem:

$$ ctxt = E_k(ptxt \oplus h(tweak)) \oplus h(tweak)$$

onde:
- $ctxt$ : criptograma recebido
- $ptxt$ : mensagem original
- $tweak$: tweak/nonce

como tal, e tendo ainda em conta que $ D_k = E^{-1}_k $ , a obtenção do texto limpo original ($ptxt$) pode ser conseguida da seguinte maneira:

$$ ptxt = D_k(ctxt \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak)) \oplus h(tweak) \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak))) \oplus h(tweak) \rightarrow ptxt \oplus h(tweak) \oplus h(tweak) \rightarrow ptxt $$

Este processo é implementado pela função auxiliar `extractMessage` que recebe como parâmetros a chave acordada (`key`), o *hash* do *tweak* (`tweak_h`) e o criptograma (`ctxt`), devolvendo o resultado de aplicar a expressão anteriormente apresentada.


In [6]:
from cryptography.hazmat.primitives.asymmetric import dh

def Receiver(conn):
    try:
        # Load long-term keys
        sign_key_dump = open("receiver_priv.pem", "rb").read()
        sign_key = load_pem_private_key(sign_key_dump, "receiver".encode("ascii"), backend = default_backend())
        
        ver_key_dump = open("emitter_pub.pem", "rb").read()
        ver_key = load_pem_public_key(ver_key_dump, backend = default_backend())
        
        # Session keys
        parameters = dh.DHParameterNumbers(P,G).parameters(backend=default_backend())

        payload = conn.recv()
        if verify_signature(ver_key, payload['sig'], payload['msg']):
            priv_key = parameters.generate_private_key()
            public_key_pem = priv_key.public_key().public_bytes(
                                                encoding=serialization.Encoding.PEM,
                                                format=serialization.PublicFormat.SubjectPublicKeyInfo)
            
            g_x = load_pem_public_key(payload['msg'], backend=default_backend())
            
            payload = sign_message(sign_key,public_key_pem)
            conn.send(payload)
            
            s_key = priv_key.exchange(g_x)[:16]
            
            payload = conn.recv()
            conn_alive = True
            
            if payload != b"0":
                while conn_alive:
                    ctxt = payload['content']
                    tag = payload['tag']
                    tweak_iv = payload['tweak_iv']
                    
                    msg = decipher_message(s_key, tweak_iv, ctxt)
                    
                    if msg:
                        conn.send(msg)
                        print("Received: " + msg.decode("utf-8"))
                        conn_alive = (msg != b"0")
                    else:
                        conn.send(b"0")
                        # Close compromised connection
                        conn_alive = False
                        
                    if conn_alive:
                        payload = conn.recv()
                    
            else:
                print("A verificação na troca de chaves falhou!")
        else:
            response = sign_message(sign_key,b"0")
            conn.send(response)
        conn.close()
    except Exception as e:
        print("Receiver:" + e)
        print("Receiver: Erro na decifragem")

O estabelecimento de uma conexão entre os dois intervenientes (`Emitter` e `Receiver`)  é feita invocando o método `auto` da classe `Connection` que cria uma comunicação síncrona entre ambos. 
Este sincronismo é exigido pelo protocolo de troca de chaves (Diffie Hellman)

In [ ]:
Connection(Emitter, Receiver, timeout=10).auto()